# Matrix Calculus Pset 2 Solutions

These is the computational notebook accompanying the pset 2 solutions for *Matrix Calculus*, MIT IAP 2023.

# Problem 1(c)

## f(p)

First, let's translate $f(p)$ to Julia.  Recall that this is defined by $f(p) = \left(c^{T}A(p)^{-1}b\right)^{2}$ where
$$
A(p)=\left(\begin{array}{ccccc}
a_{1} & p_{1}\\
p_{1} & a_{2} & p_{2}\\
 & p_{2} & \ddots & \ddots\\
 &  & \ddots & a_{n-1} & p_{n-1}\\
 &  &  & p_{n-1} & a_{n}
\end{array}\right),
$$
for some constant vectors $a,b,c$.

Let's make $f$ for $n=5$, for example.  We'll use Julia's handy [`LinearAlgebra.SymTridiagonal` type](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.SymTridiagonal) to make $A(p)$ first:

In [1]:
n = 5

# some random vectors
a = rand(-9:9, n)
b = rand(-9:9, n)
c = rand(-9:9, n)

using LinearAlgebra # for SymTridiagonal

A(p) = SymTridiagonal(promote(a, p)...)

p = rand(n-1) # for example
A(p)

5×5 SymTridiagonal{Float64, Vector{Float64}}:
 -9.0       0.416908   ⋅          ⋅          ⋅ 
  0.416908  1.0       0.292966    ⋅          ⋅ 
   ⋅        0.292966  0.0        0.310202    ⋅ 
   ⋅         ⋅        0.310202  -6.0        0.485178
   ⋅         ⋅         ⋅         0.485178  -1.0

Now, $f(p)$ is simple:

In [2]:
f(p) = (c' * (A(p) \ b))^2

f(p)

13895.37546070443

Note that `A(p) \ b` in Julia automatically uses a specialized $\Theta(n)$ tridiagonal solver since `A` is the `SymTridiagonal` type.

##  adjoint/reverse-mode ∇f

Now, let's implement our "adjoint-method" or "reverse-mode" algorithm for $\nabla f$:

1. Solve the forward problem for $x = A(p)^{-1} b$
2. Solve the adjoint problem $v = A^{-1} [-2(c^T x) c]$
3. $\partial f/\partial p_k = v_k x_{k+1} + v_{k+1} x_k$

In [3]:
function ∇f(p)
    M = A(p)
    x = M \ b
    v = M \ (-2(c'x)c)
    return [ v[k]*x[k+1] + v[k+1]*x[k] for k = 1:length(p) ]
end

∇f (generic function with 1 method)

In [4]:
∇f(p)

4-element Vector{Float64}:
    8672.424124240617
 -234089.45857983222
   69396.70127620807
    8926.845054799993

## Finite-difference check

Now let's do a finite-difference check that
$$
f(p+\delta) - f(p) \approx \nabla f^T \delta
$$
for a small random $\delta$:

In [5]:
δ = randn(n-1) * 1e-8

df = f(p+δ) - f(p)

0.00016509416491317097

In [6]:
∇f(p)'δ

0.00016509415974040186

Looks good!  They match to quite a few digits!

We can be more quantitative by computing the relative error:

In [7]:
relerr(a,b) = norm(a - b) / norm(b)

relerr(df, ∇f(p)'δ)

3.1332235629796853e-8

So, the relative error is about $10^{-8}$, pretty reasonable for $\delta$ on that order.

## AD check

For fun, let's also try automatic differentiation via ForwardDiff.jl or Zygote.jl (this was **not** required):

In [8]:
import ForwardDiff, Zygote

In [9]:
relerr(ForwardDiff.gradient(f, p), ∇f(p))

2.4586950654248017e-16

Great!  They match to almost [machine precision](https://en.wikipedia.org/wiki/Machine_epsilon)!

Let's try Zygote:

In [10]:
relerr(Zygote.gradient(f, p), ∇f(p))

LoadError: Need an adjoint for constructor SymTridiagonal{Float64, Vector{Float64}}. Gradient is of type Matrix{Float64}

Whoops, Zygote doesn't know how to differentiate through the `SymTridiagonal` function.  Grr.

# Problem 4b

Our test matrix $M$ is:

In [11]:
M = [0 1 4; 1 0 1; 4 1 0]

3×3 Matrix{Int64}:
 0  1  4
 1  0  1
 4  1  0

## (i) By finite differences

Here, we want the "matrix Jacobian determinant" for a matrix-valued function $f(M)$, defined by $\det J$ where $\text{vec}(df) = J \text{vec}(M)$.

Let's do it by (forward) finite differences, column by column in $J$.

One handy trick is that if you have an $m \times n$ matrix `M` in Julia, then you can index it by row/column as `M[i,j]` **or** you can index it "linearly" as `M[i]` for $i = 1,\ldots,mn$ as if it were an $mn$-component vector, exactly corresponding to $\text{vec}(M)$ (since this is, in fact, how Julia stores the matrix in your computer's "1d" memory).  

In [12]:
function fd_jacobian(f, M, δ=1e-8)
    fM = f(M)
    Mplusδ = float.(M) # make a floating-point copy of M
    J = zeros(length(fM), length(M))
    for j = 1:length(M)
        Mplusδ[j] += δ
        fMplusδ = f(Mplusδ)
        for i = 1:length(fM)
            J[i,j] = (fMplusδ[i] - fM[i]) / δ
        end
        Mplusδ[j] = M[j] # reset for next finite difference
    end
    return J
end

fd_jacobian (generic function with 2 methods)

In [13]:
fd_jacobian(exp, M)

9×9 Matrix{Float64}:
 22.9849    7.49817  18.369     7.49817  …  18.369     5.4667   13.7714
  7.49818  11.3611    7.49818   2.1476       5.46671  10.2735    5.46671
 18.369     7.49817  22.9849    5.4667      13.7714    5.4667   18.369
  7.49818   2.1476    5.46671  11.3611       7.49818   2.14759   5.46671
  2.1476    4.37449   2.1476    4.37449      2.1476    4.37449   2.1476
  5.46671   2.1476    7.49818  10.2735   …   5.46671   2.1476    7.49818
 18.369     5.4667   13.7714    7.49817     22.9849    7.49817  18.369
  5.46671  10.2735    5.46671   2.1476       7.49818  11.3611    7.49818
 13.7714    5.46671  18.369     5.46671     18.369     7.49817  22.9849

Now, let's use that to compute the Jacobian determinant for $f \in {\exp, \sin}$.

Note that, in Julia, you can compute the matrix exponential or sine simply by `exp(M)` or `sin(M)` respectively — these are *not* the elementwise functions.   (In Scipy or Matlab you would need `expm` and `sinm`.)

In [14]:
Jexp_fd = det(fd_jacobian(exp, M))

939.0318094614613

In [15]:
Jsq_fd = det(fd_jacobian(M -> M^2, M))

4096.000162495347

In [16]:
Jsin_fd = det(fd_jacobian(sin, M))

8.413470707960181e-6

So, we get **about 939.0, 4096, and -8.41×10⁻⁶**, respectively.

We'll check these numbers against the analytical formula in part (ii).

## (i) By automatic differentiation

Alternatively, we can try to do this by AD:

In [17]:
import ForwardDiff, Zygote

We'll start with the easy case of $M^2$:

In [18]:
Jsq = ForwardDiff.jacobian(M -> M^2, M)

9×9 Matrix{Int64}:
 0  1  4  1  0  0  4  0  0
 1  0  1  0  1  0  0  4  0
 4  1  0  0  0  1  0  0  4
 1  0  0  0  1  4  1  0  0
 0  1  0  1  0  1  0  1  0
 0  0  1  4  1  0  0  0  1
 4  0  0  1  0  0  0  1  4
 0  4  0  0  1  0  1  0  1
 0  0  4  0  0  1  4  1  0

In [19]:
det(Jsq)

4096.0

Unfortunately, `ForwardDiff` doesn't support the matrix-exponential function because the Julia linear-algebra library's `exp` function cannot handle matrices full of dual numbers.

(See also the discussion in [ForwardDiff.jl#174](https://github.com/JuliaDiff/ForwardDiff.jl/issues/174).)

In [20]:
ForwardDiff.jacobian(exp, M)

LoadError: MethodError: no method matching exp(::Matrix{ForwardDiff.Dual{ForwardDiff.Tag{typeof(exp), Int64}, Int64, 9}})
[0mClosest candidates are:
[0m  exp([91m::Union{Float16, Float32, Float64}[39m) at special/exp.jl:326
[0m  exp([91m::StridedMatrix{var"#s885"} where var"#s885"<:Union{Float32, Float64, ComplexF32, ComplexF64}[39m) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/dense.jl:569
[0m  exp([91m::StridedMatrix{var"#s885"} where var"#s885"<:Union{Integer, Complex{<:Integer}}[39m) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/dense.jl:570
[0m  ...

Let's try the [Zygote.jl package](https://github.com/FluxML/Zygote.jl)

In [21]:
Zygote.jacobian(exp, M)

LoadError: Mutating arrays is not supported -- called setindex!(Matrix{Float64}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


Nope!

Maybe if we try to implement the matrix exponential ourselves using eigenvalues/vectors, which should be good for symmetric matrices at least:

In [22]:
function fM(f, M)
    F = eigen(M)
    return F.vectors * Diagonal(f.(F.values)) / F.vectors
end

fM (generic function with 1 method)

In [23]:
Zygote.jacobian(M -> fM(exp, M), M)[1]

9×9 Matrix{Float64}:
 22.9849   0.0  0.0  14.9964   2.1476   0.0  36.7379  10.9334   13.7714
  7.49818  0.0  0.0  13.5087   4.37449  0.0  12.9649  12.4211    5.46671
 18.369    0.0  0.0  12.9649   2.1476   0.0  36.7562  12.9649   18.369
  7.49818  0.0  0.0  13.5087   4.37449  0.0  12.9649  12.4211    5.46671
  2.1476   0.0  0.0   8.74898  4.13663  0.0   4.2952   8.74898   2.1476
  5.46671  0.0  0.0  12.4211   4.37449  0.0  12.9649  13.5087    7.49818
 18.369    0.0  0.0  12.9649   2.1476   0.0  36.7562  12.9649   18.369
  5.46671  0.0  0.0  12.4211   4.37449  0.0  12.9649  13.5087    7.49818
 13.7714   0.0  0.0  10.9334   2.1476   0.0  36.7379  14.9964   22.9849

Success!  Hmm, it doesn't look right though — those columns of zeros look wrong.

It turns out that this is a [bug in Zygote](https://github.com/FluxML/Zygote.jl/issues/1369) for computing derivatives of eigenvalues when the matrix happen to be real-symmetric/Hermitian.

We can work around the problem by slightly perturbing M to make it nonsymmetric:

In [24]:
Jexp = Zygote.jacobian(M -> fM(exp, M), M + [0 0 1e-13; 0 0 0; 0 0 0])[1]

9×9 Matrix{Float64}:
 22.9849    7.49818  18.369     7.49818  …  18.369     5.46671  13.7714
  7.49818  11.3611    7.49818   2.1476       5.46671  10.2735    5.46671
 18.369     7.49818  22.9849    5.46671     13.7714    5.46671  18.369
  7.49818   2.1476    5.46671  11.3611       7.49818   2.1476    5.46671
  2.1476    4.37449   2.1476    4.37449      2.1476    4.37449   2.1476
  5.46671   2.1476    7.49818  10.2735   …   5.46671   2.1476    7.49818
 18.369     5.46671  13.7714    7.49818     22.9849    7.49818  18.369
  5.46671  10.2735    5.46671   2.1476       7.49818  11.3611    7.49818
 13.7714    5.46671  18.369     5.46671     18.369     7.49818  22.9849

In [25]:
det(Jexp)

939.0589449936064

In [26]:
Jsq = Zygote.jacobian(M -> M^2, M)[1]

9×9 Matrix{Int64}:
 0  1  4  1  0  0  4  0  0
 1  0  1  0  1  0  0  4  0
 4  1  0  0  0  1  0  0  4
 1  0  0  0  1  4  1  0  0
 0  1  0  1  0  1  0  1  0
 0  0  1  4  1  0  0  0  1
 4  0  0  1  0  0  0  1  4
 0  4  0  0  1  0  1  0  1
 0  0  4  0  0  1  4  1  0

In [27]:
det(Jsq)

4096.0

In [28]:
Jsin = Zygote.jacobian(M -> fM(sin, M), M + [0 0 1e-13; 0 0 0; 0 0 0])[1]

9×9 Matrix{Float64}:
 -0.327595   -0.0100503   0.107194   …   0.107194   -0.0369293  -0.111661
 -0.0100503  -0.180199   -0.0100503     -0.0369293   0.14325    -0.0369293
  0.107194   -0.0100503  -0.327595      -0.111661   -0.0369293   0.107194
 -0.0100503   0.0357343  -0.0369293     -0.0100503   0.0357343  -0.0369293
  0.0357343  -0.189917    0.0357343      0.0357343  -0.189917    0.0357343
 -0.0369293   0.0357343  -0.0100503  …  -0.0369293   0.0357343  -0.0100503
  0.107194   -0.0369293  -0.111661      -0.327595   -0.0100503   0.107194
 -0.0369293   0.14325    -0.0369293     -0.0100503  -0.180199   -0.0100503
 -0.111661   -0.0369293   0.107194       0.107194   -0.0100503  -0.327595

In [29]:
det(Jsin)

8.413463127258828e-6

So, we eventually got determinants of **about 939.1, exactly 4096, and about -8.41×10⁻⁶**, respectively, pretty close to our finite-difference answers!

## (ii) Analytical formula

Now we'll compare to the analytical formula:
$$
\frac{\prod_{i < j} |f(\lambda_i)-
f(\lambda_j)|^2}{\prod_{i < j} |\lambda_i-\lambda_j|^2}
\prod_i f'(\lambda_i)
$$
or equivalently
$$
\prod_{\lambda_i \ne \lambda_j} \frac{|f(\lambda_i)-
f(\lambda_j)|}{|\lambda_i-\lambda_j|}
\prod_i f'(\lambda_i)
$$

with a little help from the [`product` iterator](https://docs.julialang.org/en/v1/base/iterators/#Base.Iterators.product):

In [30]:
function Jdet_analytic(f, M)
    λ = eigvals(M)
    return prod(λ -> ForwardDiff.derivative(f, λ), λ) *
           prod(Iterators.product(λ, λ)) do ((λᵢ,λⱼ))
              λᵢ ≠ λⱼ ? abs((f(λᵢ)-f(λⱼ)) / (λᵢ-λⱼ)) : 1.0
           end
end

Jdet_analytic (generic function with 1 method)

In [31]:
Jexp_analytic = Jdet_analytic(exp, M)

939.058944993412

In [32]:
Jsq_analytic = Jdet_analytic(M -> M^2, M)

4095.999999999958

In [33]:
Jsin_analytic = Jdet_analytic(sin, M)

8.413463127260002e-6

That is, we got determinants of **about 939.1, 4096, and -8.41×10⁻⁶**, respectively, pretty close to our answers above!

Let's compare them more quantitatively by computing the relative errors in the finite-difference approximations:

In [34]:
relerr(Jexp_fd, Jexp_analytic),
relerr(Jsq_fd, Jsq_analytic),
relerr(Jsin_fd, Jsin_analytic)

(2.8896516129601107e-5, 3.9671725682311325e-8, 9.010201939337017e-7)

So, the finite-difference results were accurate to about 5 digits, 8 digits, and 6 digits, respectively.  Not too bad!

And now computing the relative errors in the Zygote AD results:

In [35]:
relerr(det(Jexp), Jexp_analytic),
relerr(det(Jsq), Jsq_analytic),
relerr(det(Jsin), Jsin_analytic)

(2.0702054278959802e-13, 1.0214051826551544e-14, 1.3953679324874998e-13)

The Zygote results were accurate to about 13 digits, which is pretty good considering that we "fudged" $M$ by $10^{-13}$ to work around the Zygote bug for symmetric $M$.